Progetto per trovare parole adatte ad un cruciverba

Prima dobbiamo importare il file .txt con il nostro dizionario di tutte le parole. una parola per riga; e inserirle in un vettore che consenta di iterare lettera per lettera quando dovremo cercare corrispondenze

In [30]:
def importa_dizionario(percorsi_file):
    """
    Importa parole da uno o più file di dizionario.
    
    Args:
        percorsi_file: Stringa con un singolo percorso o lista di percorsi ai file dizionario
    
    Returns:
        Lista di parole dove ogni parola è rappresentata come lista di caratteri
    """
    # Se è una stringa, convertila in lista
    if isinstance(percorsi_file, str):
        percorsi_file = [percorsi_file]
    
    parole = []
    for percorso_file in percorsi_file:
        try:
            with open(percorso_file, 'r', encoding='utf-8') as f:
                parole_file = [list(riga.strip().lower()) for riga in f if riga.strip()]
                parole.extend(parole_file)
                print(f"Importate {len(parole_file)} parole da {percorso_file}")
        except FileNotFoundError:
            print(f"Attenzione: File {percorso_file} non trovato")
        except Exception as e:
            print(f"Errore durante l'importazione di {percorso_file}: {e}")
    
    # Rimuovi duplicati mantenendo l'ordine
    parole_uniche = []
    parole_viste = set()
    for parola in parole:
        parola_str = ''.join(parola)
        if parola_str not in parole_viste:
            parole_uniche.append(parola)
            parole_viste.add(parola_str)
    
    print(f"Totale parole uniche importate: {len(parole_uniche)}")
    return parole_uniche

Adesso dobbiamo creare la funzione per la ricerca mediante caratteri jolly

In [31]:
def cerca_parole(parole, pattern):
    """
    Cerca parole che corrispondono al pattern, dove '*' è un carattere jolly.
    La ricerca è case insensitive.
    Esempio: '**o**' trova parole di 5 lettere con 'o' come terza lettera.
    """
    # Converte il pattern in minuscolo per confronto case insensitive
    pattern = pattern.lower()
    
    risultati = []
    for parola in parole:
        if len(parola) != len(pattern):
            continue
        corrisponde = True
        for p_char, pat_char in zip(parola, pattern):
            if pat_char != '*' and p_char != pat_char:
                corrisponde = False
                break
        if corrisponde:
            risultati.append(''.join(parola))
    return risultati

Adesso il programma può essere utilizzato per cercare parole in un dizionario.

In [32]:
# Esempio pratico di utilizzo

# Importa parole da entrambi i dizionari
parole = importa_dizionario(['dizionario_ita.txt', 'dizionario_eng.txt'])

# Cerca una parola specifica (esempio: parole di 4 lettere che iniziano con 'c' e finiscono con 'a')
pattern = 'c**a'
risultati = cerca_parole(parole, pattern)

print(f"Parole trovate per il pattern '{pattern}':")
for i, parola in enumerate(risultati[:10]):  # Mostra solo le prime 10
    print(f"{i+1}. {parola}")

if len(risultati) > 10:
    print(f"... e altre {len(risultati) - 10} parole")

Importate 279894 parole da dizionario_ita.txt
Importate 466550 parole da dizionario_eng.txt
Importate 466550 parole da dizionario_eng.txt
Totale parole uniche importate: 734120
Parole trovate per il pattern 'c**a':
1. cada
2. cala
3. capa
4. cara
5. casa
6. cava
7. ceca
8. cela
9. cena
10. cera
... e altre 45 parole
Totale parole uniche importate: 734120
Parole trovate per il pattern 'c**a':
1. cada
2. cala
3. capa
4. cara
5. casa
6. cava
7. ceca
8. cela
9. cena
10. cera
... e altre 45 parole


In [34]:
# Usa le parole già caricate dalla cella precedente
# (se non le hai ancora caricate, esegui prima la cella 7)

patternL = '****ei'
risultatiL = cerca_parole(parole, patternL)
print(f"Parole trovate per il pattern '{patternL}':")
print(risultatiL)

Parole trovate per il pattern '****ei':
['abigei', 'acquei', 'aculei', 'agirei', 'alisei', 'amebei', 'amerei', 'andrei', 'apogei', 'aramei', 'arerei', 'ascrei', 'atenei', 'babbei', 'baggei', 'balnei', 'baltei', 'barcei', 'battei', 'berrei', 'cammei', 'carnei', 'clipei', 'cornei', 'correi', 'cortei', 'costei', 'credei', 'crocei', 'cuprei', 'dovrei', 'eitrei', 'epigei', 'eterei', 'fendei', 'ferrei', 'fremei', 'gemmei', 'giudei', 'glutei', 'idonei', 'imenei', 'ipogei', 'lattei', 'laurei', 'lignei', 'lincei', 'lintei', 'mallei', 'mazdei', 'mellei', 'mietei', 'mirtei', 'mitrei', 'morrei', 'nausei', 'nervei', 'ninfei', 'nuclei', 'occhei', 'oserei', 'pendei', 'perdei', 'pigmei', 'plebei', 'plutei', 'pompei', 'porrei', 'potrei', 'premei', 'protei', 'rendei', 'saprei', 'scalei', 'sfarei', 'spelei', 'starei', 'tarpei', 'terrei', 'tessei', 'tornei', 'trofei', 'unirei', 'userei', 'uvacei', 'vedrei', 'vendei', 'verrei', 'vitrei', 'vivrei', 'vorrei', 'adigei', 'adygei', 'alexei', 'anhwei', 'brunei',